## Extracting the entrez gene IDs in HPIDB

In [ ]:
output = open('data/hpidb2_entrez_list.txt', 'w')
with open('data/hpidb2.mitab.txt', 'r', encoding ='ISO-8859-1') as f:
    for line in f:
        items = line.strip().split('\t')
        if '9606' not in items[9]:
            continue
        if 'entrez' in items[0]:
            entrez = items[0].split('locuslink:')[1]
            output.write(entrez + '\n')
output.close()

## After converting entrez to uniprot ids, making a dict from entrez to uniprot

In [ ]:
entrez2uniprot = {}
with open('data/hpidb2_entrez_list_uniprot.txt', 'r') as f:
    next(f)
    for line in f:
        items = line.strip().split()
        if items[3] != "reviewed":
            continue
        if ',' not in items[0]:
            entrez = items[0]
            if entrez not in entrez2uniprot:
                entrez2uniprot[entrez] = set()
            entrez2uniprot[entrez].add(items[1])
        else:
            for entrez in items[0].split(','):
                if entrez not in entrez2uniprot:
                    entrez2uniprot[entrez] = set()
                entrez2uniprot[entrez].add(items[1])
print(len(entrez2uniprot))

## Generating the hpidb2.score.txt file used in train.ipynb

In [ ]:
hpidb_uniprots = set()
output = open('data/hpidb2.score.txt', 'w')
with open('data/hpidb2.mitab.txt', 'r', encoding ='ISO-8859-1') as f:
    for line in f:
        items = line.strip().split('\t')
        if '9606' not in items[9] or '-' == items[len(items)-1]:
            continue
        taxon = items[10].split('(')[0].split(':')[1]
        if 'uniprotkb:' in items[0]:
            uniprot = items[0].split('uniprotkb:')[1]
            if '-' in uniprot: # Checking the isoforms
                if '-1' in uniprot:
                    uniprot = uniprot.split('-1')[0]
                else:
                    continue
            output.write('%s\t%s\t%s\n' % (uniprot, taxon, items[len(items)-1])) # with score
            hpidb_uniprots.add(uniprot)
        elif 'entrez' in items[0]:
            entrez = items[0].split('locuslink:')[1]
            if entrez in entrez2uniprot:
                for uniprot in entrez2uniprot[entrez]:
                    output.write('%s\t%s\t%s\n' % (uniprot, taxon, items[len(items)-1])) # with score
                    hpidb_uniprots.add(uniprot)
output.close()